In [37]:
from utils import *
import rich
from pydantic import BaseModel, Field
from typing import Literal, List, Optional
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [8]:
load_dotenv()

file_list = get_file_paths('output', file_pattern='.txt')
dfs = [pd.read_csv(file, sep='\t') for file in file_list]
dfs = [df.drop_duplicates(subset=["description"]) for df in dfs]
dfs = [ df.assign(description=df['pubDate'] + ': ' + df['description']) for df in dfs ]
docs = [df_to_docs(df, content_column='description', metadata_columns=['link', 'guid', 'type', 'id', 'sponsored', 'pubDate']) for df in dfs]
len(docs)


123

In [9]:
docs = [doc[:5] for doc in docs]
len(docs)

123

In [10]:
joined_contents = [".".join(doc.page_content for doc in sublist) for sublist in docs]
len(joined_contents)
joined_contents = joined_contents[:5]

In [11]:
def extract_info_from_rss(all_text):
    # all_text = ". ".join(doc.page_content for doc in docs)

    # Define a rich persona in the system message with added expertise
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp and insightful trader assistant of a hedge fund.
    You provide clear, concise, and actionable insights based on news feeds focusing on sectors rotation.
    In addition, for each sector provide outlook, primary catalyst, and trading insights.
    In addition, Describe more pro-cyclical and geopolitical shift if any.
    In addition, list bullish and bearish sectors/companies to look at into 2026.
    In addition, provide TACO (Trump Always Chickens Out) trade if any.
    Maintain a friendly, confident, and professional tone, making complex concepts accessible and useful."""
    )

    # Define the human message template with instructions and JSON schema
    human_message = HumanMessagePromptTemplate.from_template(
        """Extract the information required from the news feed below.


    News feed:
    {news_feed}

    """
    )
    # Create the chat prompt template
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])


    llm = ChatOpenAI(temperature=0)
    chain = chat_prompt | llm | StrOutputParser()

    result = chain.invoke({"news_feed": all_text})
    return(result)

In [ ]:
infos = [extract_info_from_rss(doc) for doc in joined_contents]
[rich.print(info) for info in infos]

In [18]:
def extract_info_from_rss(all_text):

    # Define the Pydantic model for structured output
    class Sector(BaseModel):
        Date: str = Field(..., description='The publication date of the news')
        Name: Literal[
            "Commercial Services",
            "Communications",
            "Consumer Durables",
            "Consumer Non-Durables",
            "Consumer Services",
            "Distribution Services",
            "Electronic Technology",
            "Energy Minerals",
            "Finance",
            "Health Services",
            "Health Technology",
            "Industrial Services",
            "Non-Energy Minerals",
            "Process Industries",
            "Producer Manufacturing",
            "Retail Trade",
            "Technology Services",
            "Transportation",
            "Utilities"
            ] = Field(..., description="Name of the sector")
        Outlook: str = Field(..., description='A sector outlook describes the expected future performance and conditions based on data, trends, and risks.')
        Catalyst: str = Field(..., description="A primary catalyst of a sector is the single most influential force that is expected to drive major change—positive or negative—across an entire industry.")
        Trading_insights: str = Field(..., description="Practical, actionable interpretation of the sector’s expected future conditions—something that can guide investment or trading decisions.")
        Direction_momentum: Literal["Strength", "Weakness", "Volatility", "Rotation in favor", "Rotation out of favor"] = Field(..., description="The direction of momentum of a sector is the prevailing trend in how that sector’s prices, performance, and investor sentiment are moving over a given period.")
        Sector_vs_market_position: Literal['Leading', 'Lagging', 'In Line'] = Field(..., description="Relative positioning of a sector to the overall market is the sector’s performance and strength compared to the broader market, showing whether it is leading, lagging, or moving in line with the market trend.")
    
    
    # Create the base JSON output parser
    base_parser = JsonOutputParser(pydantic_object=Sector)

    # Create the output-fixing parser wrapping the base parser and using an LLM to fix errors
    llm_for_fixing = ChatOpenAI(temperature=0)
    fixing_parser = OutputFixingParser.from_llm(parser=base_parser, llm=llm_for_fixing)

    # Define a rich persona in the system message with added expertise
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp and insightful trader assistant skilled is deriving sector trading insights from news."""
    )

    # Define the human message template with instructions and JSON schema
    human_message = HumanMessagePromptTemplate.from_template(
        """Extract information required.


    News feed:
    {news_feed}

    """
    )

    # Create the chat prompt template
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # Compose the chain: prompt -> LLM -> fixing parser
    llm = ChatOpenAI(temperature=0)
    chain = chat_prompt | llm | fixing_parser
    # all_text = ". ".join(doc.page_content for doc in docs)
    result = chain.invoke({"news_feed": all_text})
    return result

# extract_info_from_rss(joined_contents[0])



In [19]:
infos = [extract_info_from_rss(doc) for doc in joined_contents]
[rich.print(info) for info in infos]

{
    'Date': '',
    'Name': '',
    'Outlook': '',
    'Catalyst': '',
    'Trading_insights': '',
    'Direction_momentum': '',
    'Sector_vs_market_position': ''
}

{
    'Date': '2022-08-15',
    'Name': 'Technology Services',
    'Outlook': 'Positive outlook for the technology sector with new product launches and strong market 
penetration.',
    'Catalyst': 'New device launches and major company partnerships driving growth and market activity.',
    'Trading_insights': 'Consider potential uptick in sales and stock performance for companies like Apple, Amazon,
and Figma.',
    'Direction_momentum': 'Strength',
    'Sector_vs_market_position': 'Leading'
}

{
    'Date': '2022-10-20',
    'Name': 'Finance',
    'Outlook': 'Positive outlook for the finance sector with expected growth in investments and financial 
services.',
    'Catalyst': 'Upcoming regulatory changes favoring financial institutions and easing restrictions.',
    'Trading_insights': 'Opportunities for investment in fintech companies and traditional banking institutions.',
    'Direction_momentum': 'Strength',
    'Sector_vs_market_position': 'Leading'
}

{
    'Date': '2022-01-25',
    'Name': 'Finance',
    'Outlook': 'Positive outlook for the finance sector with expected growth in the upcoming months.',
    'Catalyst': 'Anticipated increase in interest rates by the Federal Reserve leading to higher profitability for 
financial institutions.',
    'Trading_insights': 'Investors are advised to consider financial stocks for potential gains in the current 
market conditions.',
    'Direction_momentum': 'Strength',
    'Sector_vs_market_position': 'Leading'
}

{
    'Date': '2022-01-15',
    'Name': 'Finance',
    'Outlook': 'Positive outlook for the financial sector with potential regulatory changes',
    'Catalyst': 'Potential discussions on banking regulations due to rejection by U.S. banks',
    'Trading_insights': 'Market sentiment positive after winning session, influencing sector performances',
    'Direction_momentum': 'Volatility',
    'Sector_vs_market_position': 'Leading'
}

[None, None, None, None, None]

In [24]:
def extract_info_from_rss(all_text):



    # Keep your existing Sector model unchanged
    class Sector(BaseModel):
        Date: str = Field(..., description='The publication date of the news')
        Name: Literal[
            "Commercial Services",
            "Communications",
            "Consumer Durables",
            "Consumer Non-Durables",
            "Consumer Services",
            "Distribution Services",
            "Electronic Technology",
            "Energy Minerals",
            "Finance",
            "Health Services",
            "Health Technology",
            "Industrial Services",
            "Non-Energy Minerals",
            "Process Industries",
            "Producer Manufacturing",
            "Retail Trade",
            "Technology Services",
            "Transportation",
            "Utilities"
            ] = Field(..., description="Name of the sector")
        Outlook: str = Field(..., description='A sector outlook describes the expected future performance and conditions based on data, trends, and risks.')
        Catalyst: str = Field(..., description="A primary catalyst of a sector is the single most influential force that is expected to drive major change—positive or negative—across an entire industry.")
        Trading_insights: str = Field(..., description="Practical, actionable interpretation of the sector's expected future conditions—something that can guide investment or trading decisions.")
        Direction_momentum: Literal["Strength", "Weakness", "Volatility", "Rotation in favor", "Rotation out of favor"] = Field(..., description="The direction of momentum of a sector is the prevailing trend in how that sector's prices, performance, and investor sentiment are moving over a given period.")
        Sector_vs_market_position: Literal['Leading', 'Lagging', 'In Line'] = Field(..., description="Relative positioning of a sector to the overall market is the sector's performance and strength compared to the broader market, showing whether it is leading, lagging, or moving in line with the market trend.")

    # New wrapper model for multiple sectors
    class MultiSectorAnalysis(BaseModel):
        """Analysis of multiple sectors from news feed."""
        sectors: List[Sector] = Field(
            ...,
            description="List of ALL relevant sectors found in the news (1-8 max). Only include sectors with clear evidence.",
            min_items=1,
            max_items=8
        )
    
    # 1. Create messages WITHOUT partial first
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp trader assistant. Analyze the FULL news feed and extract **ALL relevant sectors** mentioned.

    CRITICAL RULES:
    - Output ONLY valid JSON matching this schema: {schema}
    - Extract 1-8 sectors with clear news evidence
    - Each sector must use a Name from the enum
    - Fill ALL required fields for each sector
    - No duplicate sectors
    - No extra text outside JSON"""
    )

    human_message = HumanMessagePromptTemplate.from_template(
        """News feed:
    {news_feed}"""
    )

    # 2. Create ChatPromptTemplate
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # 3. NOW partial with schema (after creation)
    base_parser = JsonOutputParser(pydantic_object=MultiSectorAnalysis)
    chat_prompt = chat_prompt.partial(schema=base_parser.get_format_instructions())

    # 4. Build chain
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    chain = chat_prompt | llm.with_structured_output(MultiSectorAnalysis)

    # Invoke
    result = chain.invoke({"news_feed": all_text})
    # results = [chain.invoke({"news_feed": doc}) for doc in docs[:5]]
    return result



In [25]:
infos = [extract_info_from_rss(doc) for doc in joined_contents]
[rich.print(info) for info in infos]

MultiSectorAnalysis(
    sectors=[
        Sector(
            Date='2025-07-30T22:27:00Z',
            Name='Technology Services',
            Outlook='Positive outlook due to strong earnings performance from major companies like Microsoft.',
            Catalyst="Microsoft's achievement of becoming a $4 trillion company indicates strong growth in the 
technology sector.",
            Trading_insights='Investors may consider increasing their positions in technology stocks, particularly 
those showing strong earnings growth.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        ),
        Sector(
            Date='2025-07-30T20:20:49Z',
            Name='Technology Services',
            Outlook="Challenging outlook as Meta's Reality Labs reports significant losses.",
            Catalyst="Meta's $4.53 billion loss highlights potential risks in the technology sector, particularly 
in virtual and augmented reality.",
            Trading_insights='Caution is advised for investors in technology stocks with high exposure to 
unprofitable segments.',
            Direction_momentum='Weakness',
            Sector_vs_market_position='Lagging'
        )
    ]
)

MultiSectorAnalysis(
    sectors=[
        Sector(
            Date='2025-07-31',
            Name='Technology Services',
            Outlook='The technology services sector is expected to continue growing, driven by increasing demand 
for software solutions and digital transformation.',
            Catalyst='The rise of remote work and digital collaboration tools is a primary catalyst for growth in 
the technology services sector.',
            Trading_insights='Investors should look for opportunities in companies providing software solutions, as
demand is likely to remain strong.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        ),
        Sector(
            Date='2025-07-31',
            Name='Consumer Services',
            Outlook='The consumer services sector is showing resilience, with companies adapting to changing 
consumer behaviors and preferences.',
            Catalyst='E-commerce growth and changing consumer habits are key catalysts for the consumer services 
sector.',
            Trading_insights='Focus on companies that are successfully navigating tariff-related uncertainties and 
expanding their customer base.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        ),
        Sector(
            Date='2025-07-31',
            Name='Communications',
            Outlook='The communications sector is expected to benefit from increased internet usage and demand for 
digital content.',
            Catalyst='The growing user base of internet software platforms is a significant catalyst for the 
communications sector.',
            Trading_insights='Investors should consider companies with strong user engagement and partnerships in 
the digital content space.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        )
    ]
)

MultiSectorAnalysis(
    sectors=[
        Sector(
            Date='2025-08-01T19:02:14Z',
            Name='Finance',
            Outlook='The finance sector is facing uncertainty due to political influences on economic data and 
potential trade wars.',
            Catalyst='Political appointees influencing economic data and new tariff regimes.',
            Trading_insights='Investors should be cautious as political factors may lead to volatility in financial
markets.',
            Direction_momentum='Volatility',
            Sector_vs_market_position='Lagging'
        ),
        Sector(
            Date='2025-08-01T19:12:03Z',
            Name='Transportation',
            Outlook='The transportation sector, particularly electric vehicles, is under scrutiny due to legal 
liabilities associated with technology failures.',
            Catalyst='Legal rulings regarding liability for autonomous vehicle technology.',
            Trading_insights='Investors may want to reassess their positions in companies involved in autonomous 
vehicle technology due to increased legal risks.',
            Direction_momentum='Weakness',
            Sector_vs_market_position='Lagging'
        )
    ]
)

MultiSectorAnalysis(
    sectors=[
        Sector(
            Date='2025-08-04',
            Name='Finance',
            Outlook='The finance sector is experiencing volatility due to fluctuating job growth and trade 
tensions.',
            Catalyst='Recent trade deals and tariff announcements are influencing market conditions.',
            Trading_insights='Investors should monitor the impact of tariffs on international trade and its effects
on market stability.',
            Direction_momentum='Volatility',
            Sector_vs_market_position='In Line'
        ),
        Sector(
            Date='2025-08-04',
            Name='Transportation',
            Outlook='The transportation sector may face challenges due to increased tariffs affecting shipping 
costs.',
            Catalyst='Tariff increases on goods from India and the EU could impact transportation logistics.',
            Trading_insights='Investors should consider the potential rise in shipping costs and its effect on 
transportation stocks.',
            Direction_momentum='Weakness',
            Sector_vs_market_position='Lagging'
        )
    ]
)

MultiSectorAnalysis(
    sectors=[
        Sector(
            Date='2025-08-05',
            Name='Health Services',
            Outlook='The pharmaceutical sector is facing potential volatility due to proposed tariffs that could 
significantly increase costs.',
            Catalyst='Proposed tariffs on pharmaceuticals by the Trump administration.',
            Trading_insights='Investors should monitor the impact of tariff announcements on pharmaceutical stocks,
as increased costs may affect profitability.',
            Direction_momentum='Volatility',
            Sector_vs_market_position='In Line'
        ),
        Sector(
            Date='2025-08-05',
            Name='Finance',
            Outlook='The banking sector is experiencing scrutiny and potential shifts in customer relationships due
to political statements.',
            Catalyst='Claims of debanking against conservatives by major U.S. banks.',
            Trading_insights='Investors should be cautious about banking stocks as public sentiment and political 
factors may influence their performance.',
            Direction_momentum='Weakness',
            Sector_vs_market_position='Lagging'
        )
    ]
)

[None, None, None, None, None]

In [33]:
def extract_info_from_rss(all_text):



    class TradingEntity(BaseModel):
        """Single trading signal."""
        Date: str = Field(..., description='The publication date of the news')
        name: str = Field(..., description="Sector or company name (exact match from news)")
        note: str = Field("", description="Optional caution/note like '(with caution)'")

    class TradingSignals(BaseModel):
        """Bullish vs Bearish signals from news."""
        bullish: List[TradingEntity] = Field(
            min_items=1, max_items=8,
            description="Bullish sectors/companies with growth momentum"
        )
        bearish: List[TradingEntity] = Field(
            min_items=1, max_items=8, 
            description="Bearish sectors/companies with risks/weakness"
        )

    # Prompt - NO {schema} variable needed!
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, elite trader scanning news for actionable signals.

    From the news, extract **BULLISH** (growth/strength) vs **BEARISH** (risks/weakness) sectors/companies.

    Output as JSON array of exact names from news. Use "note" for cautions.

    Examples:
    - Bullish: "Private credit sector", note="with caution"
    - Bearish: "European carmakers"

    Rules:
    - 3-8 per list maximum
    - Only signals with clear news evidence
    - Companies + sectors both OK"""
    )

    human_message = HumanMessagePromptTemplate.from_template(
        """News feed:
    {news_feed}"""
    )

    # Create prompt (variables: only {news_feed})
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # with_structured_output AUTOMATICALLY injects schema!
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    chain = chat_prompt | llm.with_structured_output(TradingSignals)

    # Run
    # all_text = ". ".join(doc.page_content for doc in docs)
    result = chain.invoke({"news_feed": all_text})

    return result



In [34]:
[rich.print(extract_info_from_rss(cont)) for cont in joined_contents]

TradingSignals(
    bullish=[
        TradingEntity(Date='2025-07-30', name='Microsoft', note=''),
        TradingEntity(Date='2025-07-30', name='$4 trillion company', note='')
    ],
    bearish=[
        TradingEntity(Date='2025-07-30', name='Meta’s Reality Labs', note='loss of $4.53 billion'),
        TradingEntity(Date='2025-07-31', name='U.S. imports from India', note='25% duties announced'),
        TradingEntity(Date='2025-07-31', name='Federal Reserve', note='divided vote on interest rates')
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(Date='2025-07-31', name='Figma', note=''),
        TradingEntity(Date='2025-07-31', name='Amazon', note='')
    ],
    bearish=[TradingEntity(Date='2025-07-31', name='labor market', note='hiring slowdown expected')]
)

ValidationError: 1 validation error for TradingSignals
bullish
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/too_short

In [40]:
def extract_info_from_rss(all_text):


    class TradingEntity(BaseModel):
        """Detailed trading signal with conviction & catalysts."""
        Date: str = Field(..., description='The publication date of the news')
        name: str = Field(..., description="Exact sector/company name from news")
        
        # Trading conviction (strength of signal)
        conviction: Literal["High", "Medium", "Low"] = Field(
            ..., 
            description="Signal strength: High=strong evidence, Low=emerging/weak"
        )
        
        # Key catalyst driving the signal
        catalyst: str = Field(
            ..., 
            description="Primary news catalyst (1 sentence max)"
        )
        
        # Optional caution/risk
        note: Optional[str] = Field(
            None, 
            description="Risks/cautions like '(regulatory risk)' or '(overbought)'"
        )
        
        # Price action suggestion
        action: Literal["Buy", "Sell", "Watch", "Avoid"] = Field(
            ..., 
            description="Clear trading action"
        )

    class TradingSignals(BaseModel):
        """Detailed bullish/bearish signals."""
        bullish: List[TradingEntity] = Field(min_items=1, max_items=10)
        bearish: List[TradingEntity] = Field(min_items=1, max_items=10)

    # Prompt - NO {schema} variable needed!
    system_message = SystemMessagePromptTemplate.from_template(
        """Ava: Elite trader extracting HIGH-CONVICTION signals from news.

    For each signal provide:
    - name: Exact sector/company from news
    - conviction: High/Medium/Low (evidence strength)  
    - catalyst: 1-sentence key driver
    - action: Buy/Sell/Watch/Avoid
    - note: Risks/cautions (optional)

    Examples:
    - High conviction Buy: "Private credit", catalyst="3.4T→4.9T growth", note="regulatory risk"

    Only clear signals."""
    )

    human_message = HumanMessagePromptTemplate.from_template(
        """News feed:
    {news_feed}"""
    )

    # Create prompt (variables: only {news_feed})
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # with_structured_output AUTOMATICALLY injects schema!
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    chain = chat_prompt | llm.with_structured_output(TradingSignals)

    # Run
    # all_text = ". ".join(doc.page_content for doc in docs)
    result = chain.invoke({"news_feed": all_text})

    return result



In [41]:
[rich.print(extract_info_from_rss(cont)) for cont in joined_contents]

TradingSignals(
    bullish=[
        TradingEntity(
            Date='2025-07-30',
            name='Microsoft',
            conviction='High',
            catalyst="Microsoft becomes the world's second $4 trillion company following strong quarterly 
earnings.",
            note=None,
            action='Buy'
        )
    ],
    bearish=[
        TradingEntity(
            Date='2025-07-30',
            name='Meta’s Reality Labs',
            conviction='High',
            catalyst='Meta’s Reality Labs posts a significant $4.53 billion loss in the second quarter.',
            note=None,
            action='Sell'
        ),
        TradingEntity(
            Date='2025-07-31',
            name='U.S. Imports from India',
            conviction='Medium',
            catalyst='25% duties announced on imports from India may impact trade relations and sectors reliant on 
Indian goods.',
            note='Potential for increased costs and supply chain disruptions.',
            action='Watch'
        ),
        TradingEntity(
            Date='2025-07-30',
            name='Federal Reserve',
            conviction='Medium',
            catalyst="Federal Reserve's decision to keep interest rates steady indicates uncertainty in economic 
outlook.",
            note='Market volatility may arise from mixed signals.',
            action='Watch'
        )
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(
            Date='2025-07-31',
            name='Amazon',
            conviction='High',
            catalyst='Revenue topped $30 billion with new business from PepsiCo',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2025-07-31',
            name='Figma',
            conviction='High',
            catalyst='Fast-growing software with over 13 million users including major clients like Google and 
Microsoft',
            note=None,
            action='Buy'
        )
    ],
    bearish=[
        TradingEntity(
            Date='2025-07-31',
            name='Apple',
            conviction='Medium',
            catalyst='Typically slow sales quarter ahead of new device launches',
            note='Potential for lower sales impact on stock price',
            action='Watch'
        ),
        TradingEntity(
            Date='2025-07-31',
            name='Labor Market',
            conviction='Medium',
            catalyst="Expected sharp slowdown in hiring despite Fed's positive outlook",
            note='Indicates potential economic slowdown risks',
            action='Watch'
        )
    ]
)

ValidationError: 1 validation error for TradingSignals
bullish
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/too_short